In [1]:
import pandas as pd

In [2]:
jan_df = pd.read_csv('data/Jan Sold With.csv')
nov_df = pd.read_csv('data/Nov Sold With.csv')
dec_df = pd.read_csv('data/Dec Sold With.csv')
stores_df = pd.read_csv('data/stores.csv')
store_sales_df = pd.read_csv('data/Store Sales.csv')
products_df = pd.read_csv('data/products.csv')

C:\Users\alecb\AppData\Local\Temp\ipykernel_98236\1282520034.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  jan_df = pd.read_csv('data/Jan Sold With.csv')
C:\Users\alecb\AppData\Local\Temp\ipykernel_98236\1282520034.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  nov_df = pd.read_csv('data/Nov Sold With.csv')
C:\Users\alecb\AppData\Local\Temp\ipykernel_98236\1282520034.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  dec_df = pd.read_csv('data/Dec Sold With.csv')


#### I am concatting all the sales tables (nov. ,dec., and jan.) into one df called sales_df

In [3]:
sales_df = pd.concat([nov_df, dec_df, jan_df])
sales_df.head(3)

,DIV,STORE,TRANSACTION_DATE,PRIMARY_STOCKNO,PRIMARY_ITEM_SIZE,PRIMARY_ITEM_WIDTH,PRIMARY_EMP_IND,PRIMARY_SOS_IND,PRIMARY_UNITS,SECONDARY_STOCKNO,SECONDARY_ITEM_SIZE,SECONDARY_ITEM_WIDTH,SECONDARY_EMP_IND,SECONDARY_SOS_IND,SECONDARY_UNITS,PRIMARY_LINENO,SECONDARY_LINENO
0,8,12,02-NOV-22,99497860,080,M,N,Y,1,497403,060,M,N,N,1,1,3
1,8,12,02-NOV-22,497403,060,M,N,N,1,99497860,080,M,N,Y,1,3,1
2,8,12,03-NOV-22,398562,100,M,N,N,1,19133,XL,NaN,N,N,1,2,1


I am now joining the DEPARTMENT from the products table to the January sales table so I can filter on accessory or shoe. I will be joining on STOCKNO and PRIMARY_STOCKNO.

In [4]:
master_sale_df1 = pd.merge(sales_df, products_df[['STOCKNO','DEPARTMENT']], 
                               left_on = 'PRIMARY_STOCKNO', right_on = 'STOCKNO', 
                               how = 'left')

In [5]:
master_sale_df1 = master_sale_df1.rename(columns={"DEPARTMENT": "PRIMARY_DEPT"})

In [6]:
master_sale_df1.head(3)

,DIV,STORE,TRANSACTION_DATE,PRIMARY_STOCKNO,PRIMARY_ITEM_SIZE,PRIMARY_ITEM_WIDTH,PRIMARY_EMP_IND,PRIMARY_SOS_IND,PRIMARY_UNITS,SECONDARY_STOCKNO,SECONDARY_ITEM_SIZE,SECONDARY_ITEM_WIDTH,SECONDARY_EMP_IND,SECONDARY_SOS_IND,SECONDARY_UNITS,PRIMARY_LINENO,SECONDARY_LINENO,STOCKNO,PRIMARY_DEPT
0,8,12,02-NOV-22,99497860,080,M,N,Y,1,497403,060,M,N,N,1,1,3,99497860.0,7.0
1,8,12,02-NOV-22,497403,060,M,N,N,1,99497860,080,M,N,Y,1,3,1,497403.0,1.0
2,8,12,03-NOV-22,398562,100,M,N,N,1,19133,XL,NaN,N,N,1,2,1,398562.0,3.0


#### Doing the same thing but for SECONDARY_STOCKNO

In [7]:
master_sale_df = pd.merge(master_sale_df1, products_df[['STOCKNO','DEPARTMENT']], 
                               left_on = 'SECONDARY_STOCKNO', right_on = 'STOCKNO', 
                               how = 'left')

In [8]:
master_sale_df = master_sale_df.rename(columns={"DEPARTMENT": "SECONDARY_DEPT"})

#### Dropping some columns

In [9]:
master_sale_df = master_sale_df.drop(columns = ['TRANSACTION_DATE', 'PRIMARY_EMP_IND',
                                                              'SECONDARY_EMP_IND', 'PRIMARY_SOS_IND',
                                                              'SECONDARY_SOS_IND', 'PRIMARY_LINENO',
                                                              'SECONDARY_LINENO', 'STOCKNO_x',
                                                              'STOCKNO_y' ])

In [10]:
master_sale_df.head(5)

,DIV,STORE,PRIMARY_STOCKNO,PRIMARY_ITEM_SIZE,PRIMARY_ITEM_WIDTH,PRIMARY_UNITS,SECONDARY_STOCKNO,SECONDARY_ITEM_SIZE,SECONDARY_ITEM_WIDTH,SECONDARY_UNITS,PRIMARY_DEPT,SECONDARY_DEPT
0,8,12,99497860,080,M,1,497403,060,M,1,7.0,1.0
1,8,12,497403,060,M,1,99497860,080,M,1,1.0,7.0
2,8,12,398562,100,M,1,19133,XL,NaN,1,3.0,6.0
3,8,12,1497835,130,M,1,398564,070,M,1,4.0,3.0
4,8,12,1497835,130,M,1,398564,060,M,1,4.0,3.0


#### Creating a new column PRIMARY_SHOE (True or False). This will be used to distinguish a shoe from an accessory

In [11]:
for index, row in master_sale_df.iterrows():
    if row.PRIMARY_DEPT in [1, 2, 3, 4, 7]:
        master_sale_df.loc[index, 'PRIMARY_SHOE'] = 'True'
    elif row.PRIMARY_DEPT in [5, 6, 8, 9, 10, 15]:
        master_sale_df.loc[index, 'PRIMARY_SHOE'] = 'False'
    else:
        master_sale_df.loc[index, 'PRIMARY_SHOE'] = 'error'

#### Creating another column SECONDARY_SHOE that provides the same insight as PRIMARY_SHOE.

In [12]:
for index, row in master_sale_df.iterrows():
    if row.SECONDARY_DEPT in [1, 2, 3, 4, 7]:
        master_sale_df.loc[index, 'SECONDARY_SHOE'] = 'True'
    elif row.SECONDARY_DEPT in [5, 6, 8, 9, 10, 15]:
        master_sale_df.loc[index, 'SECONDARY_SHOE'] = 'False'
    else:
        master_sale_df.loc[index, 'SECONDARY_SHOE'] = 'error'

#### I want to be able to filter on location hierarchy so I am joining the stores table to the sales table using 'store'.

In [13]:
master_sale_df = pd.merge(master_sale_df, stores_df[['DIV','STORE','REGION','DISTRICT','STORE_STATE_ALPHA',
                                                     'STORE_ZIP','LATITUDE','LONGITUDE']], 
                               left_on = ('STORE', 'DIV'), right_on = ('STORE', 'DIV'), 
                               how = 'left')

#### Creating a table for shoes sold with shoes (shoe_shoe_sale), and a table for shoes sold with accessories (shoe_acces_sale)

In [14]:
shoe_shoe_sale = master_sale_df[(master_sale_df['SECONDARY_SHOE'] == 'True') 
                                      & (master_sale_df['PRIMARY_SHOE'] == 'True')]

shoe_acces_sale = master_sale_df[(master_sale_df['SECONDARY_SHOE'] == 'False')
                                      & (master_sale_df['PRIMARY_SHOE'] == 'True')]

### The following functions are used to find the top 3 shoe or accessory recomendations given a shoe's stock number. The first list contains the recomendations. The second list returns the amount of times this combonation has been purchased.

In [15]:
def shoe_recomendations(stock_num):
    
    recomendation_df = pd.DataFrame(shoe_shoe_sale[(shoe_shoe_sale['PRIMARY_STOCKNO'] == stock_num)]
                                    [['PRIMARY_STOCKNO','SECONDARY_STOCKNO']]).value_counts().reset_index()
    
    recomendation_df.columns = ['PRIMARY_STOCKNO', 'SECONDARY_STOCKNO', 'COUNT']
    
    recomendations = []
    combo_purchase_count = []
    
    for index, row in recomendation_df.head(3).iterrows():
        if row.SECONDARY_STOCKNO == stock_num:
            recomendations.append(row.PRIMARY_STOCKNO)
            combo_purchase_count.append(row.COUNT)
        else:
            recomendations.append(row.SECONDARY_STOCKNO)
            combo_purchase_count.append(row.COUNT)
    
    return recomendations, combo_purchase_count

In [16]:
def acces_recomendations(stock_num):
    
    recomendation_df = pd.DataFrame(shoe_acces_sale[(shoe_acces_sale['PRIMARY_STOCKNO'] == stock_num)]
                                    [['PRIMARY_STOCKNO','SECONDARY_STOCKNO']]).value_counts().reset_index()
    
    recomendation_df.columns = ['PRIMARY_STOCKNO', 'SECONDARY_STOCKNO', 'COUNT']
    
    recomendations = []
    combo_purchase_count = []
    
    for index, row in recomendation_df.head(3).iterrows():
        if row.SECONDARY_STOCKNO == stock_num:
            recomendations.append(row.PRIMARY_STOCKNO)
            combo_purchase_count.append(row.COUNT)
        else:
            recomendations.append(row.SECONDARY_STOCKNO)
            combo_purchase_count.append(row.COUNT)
    
    return recomendations, combo_purchase_count

In [17]:
shoe_recomendations(398913)

([398564, 398732, 398566], [3701, 660, 592])

In [18]:
acces_recomendations(398913)

([1639, 401, 434], [5034, 2289, 805])

In [19]:
shoe_shoe_sale.head(3)

,DIV,STORE,PRIMARY_STOCKNO,PRIMARY_ITEM_SIZE,PRIMARY_ITEM_WIDTH,PRIMARY_UNITS,SECONDARY_STOCKNO,SECONDARY_ITEM_SIZE,SECONDARY_ITEM_WIDTH,SECONDARY_UNITS,PRIMARY_DEPT,SECONDARY_DEPT,PRIMARY_SHOE,SECONDARY_SHOE,REGION,DISTRICT,STORE_STATE_ALPHA,STORE_ZIP,LATITUDE,LONGITUDE
0,8,12,99497860,080,M,1,497403,060,M,1,7.0,1.0,True,True,9.0,29.0,MI,483131320.0,42.6019,-82.998
1,8,12,497403,060,M,1,99497860,080,M,1,1.0,7.0,True,True,9.0,29.0,MI,483131320.0,42.6019,-82.998
3,8,12,1497835,130,M,1,398564,070,M,1,4.0,3.0,True,True,9.0,29.0,MI,483131320.0,42.6019,-82.998


#### Creating similiar functions that can look into different location hierarchies.

In [20]:
all_regions = shoe_shoe_sale.REGION.unique()
all_districts = shoe_shoe_sale.DISTRICT.unique()
all_states = shoe_shoe_sale.STORE_STATE_ALPHA.unique()
all_stores = shoe_shoe_sale.STORE.unique()

In [83]:
def shoe_recomendations2(stock_num, region = all_regions, district = all_districts,
                         state = all_states, store = all_stores):
    
    shoe_shoe = shoe_shoe_sale[(shoe_shoe_sale['REGION'].isin(region)) &
                                    (shoe_shoe_sale['DISTRICT'].isin(district)) &
                                    (shoe_shoe_sale['STORE_STATE_ALPHA'].isin(state)) &
                                    (shoe_shoe_sale['STORE'].isin(store))]
    if len(store) == 1:
        heirarchy ='store'
    elif len(state) == 1:
        heirarchy ='state'
    elif len(district) == 1:
        heirarchy ='district'
    elif len(region) == 1:
        heirarchy ='region'
    else:
        heirarchy = 'division'
    
    recomendation_df = pd.DataFrame(shoe_shoe[(shoe_shoe['PRIMARY_STOCKNO'] == stock_num)]
                                    [['PRIMARY_STOCKNO','SECONDARY_STOCKNO']]).value_counts().reset_index()
    
    recomendation_df.columns = ['PRIMARY_STOCKNO', 'SECONDARY_STOCKNO', 'COUNT']
    
    recomendations = []
    combo_purchase_count = []
    
    for index, row in recomendation_df.head(3).iterrows():
        if row.SECONDARY_STOCKNO == stock_num:
            recomendations.append(row.PRIMARY_STOCKNO)
            combo_purchase_count.append(row.COUNT)
        else:
            recomendations.append(row.SECONDARY_STOCKNO)
            combo_purchase_count.append(row.COUNT)
    
    return recomendations, combo_purchase_count, heirarchy

In [48]:
shoe_recomendations2(99497860)

([1497860, 99497858, 99498066], [70, 16, 14], 'heirarchy level = division')

In [84]:
def acces_recomendations2(stock_num, region = all_regions, district = all_districts,
                         state = all_states, store = all_stores):
    
    acces_acces = shoe_acces_sale[(shoe_acces_sale['REGION'].isin(region)) &
                                    (shoe_acces_sale['DISTRICT'].isin(district)) &
                                    (shoe_acces_sale['STORE_STATE_ALPHA'].isin(state)) &
                                    (shoe_acces_sale['STORE'].isin(store))]
    
    if len(store) == 1:
        heirarchy ='store'
    elif len(state) == 1:
        heirarchy ='state'
    elif len(district) == 1:
        heirarchy ='district'
    elif len(region) == 1:
        heirarchy ='region'
    else:
        heirarchy = 'division'
    
    recomendation_df = pd.DataFrame(acces_acces[(acces_acces['PRIMARY_STOCKNO'] == stock_num)]
                                    [['PRIMARY_STOCKNO','SECONDARY_STOCKNO']]).value_counts().reset_index()
    
    recomendation_df.columns = ['PRIMARY_STOCKNO', 'SECONDARY_STOCKNO', 'COUNT']
    
    recomendations = []
    combo_purchase_count = []
    
    for index, row in recomendation_df.head(3).iterrows():
        if row.SECONDARY_STOCKNO == stock_num:
            recomendations.append(row.PRIMARY_STOCKNO)
            combo_purchase_count.append(row.COUNT)
        else:
            recomendations.append(row.SECONDARY_STOCKNO)
            combo_purchase_count.append(row.COUNT)
    
    return recomendations, combo_purchase_count, heirarchy

In [35]:
acces_recomendations2(99497860)

heirarchy level = division


([1829, 3875, 1018], [31, 24, 22])

### I would like to apply the functions I created to the sales table (shoe&shoe sales, shoe&accessory sales). My desired result is a table with the store number, product number, 6 recomendations (3 shoes, 3 accessories), and at what heirarchy level the recomendations come from (store, region, state, etc.)

Setting up a table to be filled

In [104]:
test8 = shoe_shoe_sale.reset_index()
test8 = test8.drop(columns = ['index', 'PRIMARY_UNITS', 'SECONDARY_STOCKNO', 'SECONDARY_UNITS',
                              'SECONDARY_ITEM_SIZE', 'SECONDARY_ITEM_SIZE', 'SECONDARY_ITEM_WIDTH',
                              'SECONDARY_DEPT', 'SECONDARY_SHOE', 'STORE_ZIP',
                              'LATITUDE', 'LONGITUDE'])
test8.head(10)

,DIV,STORE,PRIMARY_STOCKNO,PRIMARY_ITEM_SIZE,PRIMARY_ITEM_WIDTH,PRIMARY_DEPT,PRIMARY_SHOE,REGION,DISTRICT,STORE_STATE_ALPHA
0,8,12,99497860,080,M,7.0,True,9.0,29.0,MI
1,8,12,497403,060,M,1.0,True,9.0,29.0,MI
2,8,12,1497835,130,M,4.0,True,9.0,29.0,MI
3,8,12,1497835,130,M,4.0,True,9.0,29.0,MI
4,8,12,398564,070,M,3.0,True,9.0,29.0,MI
5,8,12,398564,060,M,3.0,True,9.0,29.0,MI
6,8,12,1399645,010,M,4.0,True,9.0,29.0,MI
7,8,12,1399645,010,M,4.0,True,9.0,29.0,MI
8,8,12,1399645,010,M,4.0,True,9.0,29.0,MI
9,8,12,497875,060,M,3.0,True,9.0,29.0,MI


In [125]:
shoe_shoe_sale[shoe_shoe_sale['STORE']==281].shape

(2708, 20)

### The below for loop uses the shoe_recomendations2 function on each primary_stockno. It starts by looking at the store level. If it does not find 3 recomendations (with more than 1 of that combo purchased) it will move to the highest level. If it can't find a recomendation for a shoe it will return -1.<br><br>
#### This loop is pretty inefficient and will take an extremely long time to complete the entire sales table (1.76 million rows). The current version is running on 11 rows and takes approximately 40 seconds to complete. This would equate to 81 days for it to run through all 1.76 million rows.

In [100]:
for index, row in test8.head(11).iterrows():
    
    if index % 10 == 0:
        print(index)
        
    if len(shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION], [row.DISTRICT],
                            [row.STORE_STATE_ALPHA], [row.STORE])[1]) == 3 and shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION],
                                                                                                    [row.DISTRICT],[row.STORE_STATE_ALPHA],
                                                                                                    [row.STORE])[1][2] > 1:
                            
        test8.loc[index, 'rec1'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA], [row.STORE])[0][0]
        test8.loc[index, 'rec2'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA], [row.STORE])[0][1]
        test8.loc[index, 'rec3'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA], [row.STORE])[0][2]
        test8.loc[index, 'rec1_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA], [row.STORE])[1][0]
        test8.loc[index, 'rec2_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA], [row.STORE])[1][1]
        test8.loc[index, 'rec3_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA], [row.STORE])[1][2]
        test8.loc[index, 'rec_level'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA], [row.STORE])[2]
        
    elif len(shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION], [row.DISTRICT],
                            [row.STORE_STATE_ALPHA])[1]) == 3 and shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION], [row.DISTRICT],
                                                                                       [row.STORE_STATE_ALPHA])[1][2] > 1:
        test8.loc[index, 'rec1'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA])[0][0]
        test8.loc[index, 'rec2'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA])[0][1]
        test8.loc[index, 'rec3'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA])[0][2]
        test8.loc[index, 'rec1_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA])[1][0]
        test8.loc[index, 'rec2_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA])[1][1]
        test8.loc[index, 'rec3_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA])[1][2]
        test8.loc[index, 'rec_level'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT],
                                                            [row.STORE_STATE_ALPHA])[2]
        
    elif len(shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION], [row.DISTRICT])[1]) == 3 and shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                                                                                       [row.REGION],
                                                                                                                       [row.DISTRICT])[1][2] > 1:
        test8.loc[index, 'rec1'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT])[0][0]
        test8.loc[index, 'rec2'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT])[0][1]
        test8.loc[index, 'rec3'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT])[0][2]
        test8.loc[index, 'rec1_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT])[1][0]
        test8.loc[index, 'rec2_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT])[1][1]
        test8.loc[index, 'rec3_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT])[1][2]
        test8.loc[index, 'rec_level'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT])[2]
        
    elif len(shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION])[2]) == 3 and shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                                                                       [row.REGION])[1][2] > 1:
        test8.loc[index, 'rec1'] = shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION])[0][0]
        test8.loc[index, 'rec2'] = shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION])[0][1]
        test8.loc[index, 'rec3'] = shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION])[0][2]
        test8.loc[index, 'rec1_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION])[1][0]
        test8.loc[index, 'rec2_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION])[1][1]
        test8.loc[index, 'rec3_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION])[1][2]
        test8.loc[index, 'rec_level'] = shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION])[2]
        
    elif len(shoe_recomendations2(row.PRIMARY_STOCKNO)[1]) == 3:
        test8.loc[index, 'rec1'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[0][0]
        test8.loc[index, 'rec2'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[0][1]
        test8.loc[index, 'rec3'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[0][2]
        test8.loc[index, 'rec1_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[1][0]
        test8.loc[index, 'rec2_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[1][1]
        test8.loc[index, 'rec3_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[1][2]
        test8.loc[index, 'rec_level'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[2]
        
    elif len(shoe_recomendations2(row.PRIMARY_STOCKNO)[1]) == 2:
        test8.loc[index, 'rec1'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[0][0]
        test8.loc[index, 'rec2'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[0][1]
        test8.loc[index, 'rec3'] = -1
        test8.loc[index, 'rec1_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[1][0]
        test8.loc[index, 'rec2_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[1][1]
        test8.loc[index, 'rec3_count'] = -1
        test8.loc[index, 'rec_level'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[2]
        
    elif len(shoe_recomendations2(row.PRIMARY_STOCKNO)[1]) == 1:
        test8.loc[index, 'rec1'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[0][0]
        test8.loc[index, 'rec2'] = -1
        test8.loc[index, 'rec3'] = -1
        test8.loc[index, 'rec1_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[1][0]
        test8.loc[index, 'rec2_count'] = -1
        test8.loc[index, 'rec3_count'] = -1
        test8.loc[index, 'rec_level'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[2]
        
    else:
        test8.loc[index, 'rec1'] = -1
        test8.loc[index, 'rec2'] = -1
        test8.loc[index, 'rec3'] = -1
        test8.loc[index, 'rec1_count'] = -1
        test8.loc[index, 'rec2_count'] = -1
        test8.loc[index, 'rec3_count'] = -1
        test8.loc[index, 'rec_level'] = -1

0
10


In [98]:
test8.head(10)

,DIV,STORE,PRIMARY_STOCKNO,PRIMARY_ITEM_SIZE,PRIMARY_ITEM_WIDTH,PRIMARY_DEPT,PRIMARY_SHOE,REGION,DISTRICT,STORE_STATE_ALPHA,rec1,rec2,rec3,rec1_count,rec2_count,rec3_count,rec_level
0,8,12,99497860,080,M,7.0,True,9.0,29.0,MI,1497860.0,99497858.0,99498066.0,70.0,16.0,14.0,division
1,8,12,497403,060,M,1.0,True,9.0,29.0,MI,498066.0,497419.0,398564.0,312.0,92.0,85.0,division
2,8,12,1497835,130,M,4.0,True,9.0,29.0,MI,1497800.0,1497878.0,1497721.0,17.0,15.0,13.0,division
3,8,12,1497835,130,M,4.0,True,9.0,29.0,MI,1497800.0,1497878.0,1497721.0,17.0,15.0,13.0,division
4,8,12,398564,070,M,3.0,True,9.0,29.0,MI,398913.0,399645.0,1497835.0,3.0,2.0,2.0,store
5,8,12,398564,060,M,3.0,True,9.0,29.0,MI,398913.0,399645.0,1497835.0,3.0,2.0,2.0,store
6,8,12,1399645,010,M,4.0,True,9.0,29.0,MI,1398564.0,1398566.0,1398329.0,40.0,32.0,32.0,division
7,8,12,1399645,010,M,4.0,True,9.0,29.0,MI,1398564.0,1398566.0,1398329.0,40.0,32.0,32.0,division
8,8,12,1399645,010,M,4.0,True,9.0,29.0,MI,1398564.0,1398566.0,1398329.0,40.0,32.0,32.0,division
9,8,12,497875,060,M,3.0,True,9.0,29.0,MI,497877.0,497802.0,497854.0,30.0,27.0,25.0,division


In [105]:
for index, row in test8.head(11).iterrows():
        
    if len(shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION], [row.DISTRICT])[1]) == 3 and shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                                                                                       [row.REGION],
                                                                                                                       [row.DISTRICT])[1][2] > 1:
        test8.loc[index, 'rec1'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT])[0][0]
        test8.loc[index, 'rec2'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT])[0][1]
        test8.loc[index, 'rec3'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT])[0][2]
        test8.loc[index, 'rec1_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT])[1][0]
        test8.loc[index, 'rec2_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT])[1][1]
        test8.loc[index, 'rec3_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT])[1][2]
        test8.loc[index, 'rec_level'] = shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                            [row.REGION], [row.DISTRICT])[2]
        
    elif len(shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION])[2]) == 3 and shoe_recomendations2(row.PRIMARY_STOCKNO,
                                                                                                       [row.REGION])[1][2] > 1:
        test8.loc[index, 'rec1'] = shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION])[0][0]
        test8.loc[index, 'rec2'] = shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION])[0][1]
        test8.loc[index, 'rec3'] = shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION])[0][2]
        test8.loc[index, 'rec1_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION])[1][0]
        test8.loc[index, 'rec2_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION])[1][1]
        test8.loc[index, 'rec3_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION])[1][2]
        test8.loc[index, 'rec_level'] = shoe_recomendations2(row.PRIMARY_STOCKNO, [row.REGION])[2]
        
    elif len(shoe_recomendations2(row.PRIMARY_STOCKNO)[1]) == 3:
        test8.loc[index, 'rec1'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[0][0]
        test8.loc[index, 'rec2'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[0][1]
        test8.loc[index, 'rec3'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[0][2]
        test8.loc[index, 'rec1_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[1][0]
        test8.loc[index, 'rec2_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[1][1]
        test8.loc[index, 'rec3_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[1][2]
        test8.loc[index, 'rec_level'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[2]
        
    elif len(shoe_recomendations2(row.PRIMARY_STOCKNO)[1]) == 2:
        test8.loc[index, 'rec1'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[0][0]
        test8.loc[index, 'rec2'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[0][1]
        test8.loc[index, 'rec3'] = -1
        test8.loc[index, 'rec1_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[1][0]
        test8.loc[index, 'rec2_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[1][1]
        test8.loc[index, 'rec3_count'] = -1
        test8.loc[index, 'rec_level'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[2]
        
    elif len(shoe_recomendations2(row.PRIMARY_STOCKNO)[1]) == 1:
        test8.loc[index, 'rec1'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[0][0]
        test8.loc[index, 'rec2'] = -1
        test8.loc[index, 'rec3'] = -1
        test8.loc[index, 'rec1_count'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[1][0]
        test8.loc[index, 'rec2_count'] = -1
        test8.loc[index, 'rec3_count'] = -1
        test8.loc[index, 'rec_level'] = shoe_recomendations2(row.PRIMARY_STOCKNO)[2]
        
    else:
        test8.loc[index, 'rec1'] = -1
        test8.loc[index, 'rec2'] = -1
        test8.loc[index, 'rec3'] = -1
        test8.loc[index, 'rec1_count'] = -1
        test8.loc[index, 'rec2_count'] = -1
        test8.loc[index, 'rec3_count'] = -1
        test8.loc[index, 'rec_level'] = -1

In [114]:
test8.head(11)

,DIV,STORE,PRIMARY_STOCKNO,PRIMARY_ITEM_SIZE,PRIMARY_ITEM_WIDTH,PRIMARY_DEPT,PRIMARY_SHOE,REGION,DISTRICT,STORE_STATE_ALPHA,rec1,rec2,rec3,rec1_count,rec2_count,rec3_count,rec_level
0,8,12,99497860,080,M,7.0,True,9.0,29.0,MI,1497860.0,99497858.0,99498066.0,70.0,16.0,14.0,division
1,8,12,497403,060,M,1.0,True,9.0,29.0,MI,498066.0,497419.0,398564.0,312.0,92.0,85.0,division
2,8,12,1497835,130,M,4.0,True,9.0,29.0,MI,1497800.0,1497878.0,1497721.0,17.0,15.0,13.0,division
3,8,12,1497835,130,M,4.0,True,9.0,29.0,MI,1497800.0,1497878.0,1497721.0,17.0,15.0,13.0,division
4,8,12,398564,070,M,3.0,True,9.0,29.0,MI,398913.0,398504.0,398566.0,27.0,9.0,7.0,district
5,8,12,398564,060,M,3.0,True,9.0,29.0,MI,398913.0,398504.0,398566.0,27.0,9.0,7.0,district
6,8,12,1399645,010,M,4.0,True,9.0,29.0,MI,1398564.0,1398566.0,1398329.0,40.0,32.0,32.0,division
7,8,12,1399645,010,M,4.0,True,9.0,29.0,MI,1398564.0,1398566.0,1398329.0,40.0,32.0,32.0,division
8,8,12,1399645,010,M,4.0,True,9.0,29.0,MI,1398564.0,1398566.0,1398329.0,40.0,32.0,32.0,division
9,8,12,497875,060,M,3.0,True,9.0,29.0,MI,497877.0,497802.0,497854.0,30.0,27.0,25.0,division


In [119]:
test8[test8['STORE_STATE_ALPHA']=='TN']['STORE'].unique()

array([ 128,  281,  492,  583,  782,  814,  975,  995, 1055, 1111, 1245,
       1248, 1284, 1297, 1344, 1399, 1408, 1417, 1599, 1603, 1662, 1697,
       1750, 3001, 3029, 7028, 7082, 7084, 7197], dtype=int64)

In [118]:
len(test8[test8['STORE_STATE_ALPHA']=='TN']['STORE'].unique())

29

29 stores in TN. A little over an hour to run the above loop on one store. could split this up and just look at TN stores.

In [124]:
len(test8[test8['STORE']==583])

1124

In [110]:
test8.STORE.unique()

array([  12,   11,   17, ..., 7215, 7216, 7217], dtype=int64)